# Dependencies

In [ ]:
import numpy as np
import scipy as sp
import scipy.io as spio
import torch
import pulp
import matplotlib.pyplot as plt
import time
from tqdm import trange, tqdm

# Definition of helper functions

In [ ]:
def generate_simple_example(c, p):
    T = 4
    delta = c * 10
    Delta = c * 2
    m = 2
    n = 2
    X = np.ones((T, m, 1))
    Y = np.ones((T, n, 1))
    X[:, 0, 0] = -(delta + Delta) / 2 * X[:, 0, 0]
    X[:, 1, 0] = -X[:, 0, 0]
    Y[:2, 0, 0] = - delta / 2 * Y[:2, 0, 0]
    Y[2:, 0, 0] = delta / 2 * Y[2:, 0, 0]
    Y[:, 1, 0] = -Y[:, 0, 0]
    return X, Y

def generate_extreme_example(c, p):
    T = 100
    m = 2
    n = 2
    X = np.ones((T, m, 1))
    Y = np.ones((T, n, 1))
    X[:, 0, 0] = -10
    X[:, 1, 0] = 10
    Y[:(T // 2), 0, 0] = -10
    Y[(T // 2):, 0, 0] = 10
    Y[:(T // 2), 1, 0] = 10
    Y[(T // 2):, 1, 0] = -10
    return X, Y

def d(x, y, c, p):
    x_nan, y_nan = np.any(np.isnan(x)), np.any(np.isnan(y))
    if x_nan and y_nan:
        return 0
    elif x_nan or y_nan:
        return c / 2**(1 / p)
    else:
        return np.minimum(c, np.linalg.norm(x - y, ord=p)) # TODO: Is this correct?
    
def calculate_distance_matrices(X, Y, c, p):
    T, m, _ = X.shape
    T, n, _ = Y.shape
    D = np.zeros((T, m + 1, n + 1))
    for t in range(T):
        for i in range(m + 1):
            for j in range(n + 1):
                if i == m:
                    x = np.nan
                else:
                    x = X[t, i, :]
                if j == n:
                    y = np.nan
                else:
                    y = Y[t, j, :]
                D[t, i, j] = d(x, y, c, p)**p
    return D

def generate_trajectories(T, m, l, birth_death_prob=0.5):
    tau = l / T
    F = np.kron(np.eye(2), np.array([[1, tau], [0, 1]]))
    Q = np.kron(np.eye(2), np.array([[tau**3 / 3, tau**2 / 2], [tau**2 / 2, tau]]))
    X = np.zeros((T, m, 4))
    X[:] = np.nan
    for i in range(m):
        t_birth, t_death = 0, 0
        while t_birth == t_death or t_birth < 0 or t_death >= T:
            t_birth = np.random.geometric(birth_death_prob) - 1
            t_death = T - (np.random.geometric(birth_death_prob) - 1)
            if t_birth > t_death:
                t_birth, t_death = t_death, t_birth
        X[t_birth, i, (0, 2)] = np.random.multivariate_normal(np.ones(2), np.eye(2))
        X[t_birth, i, (1, 3)] = np.random.multivariate_normal(-X[t_birth, i, (0, 2)], np.eye(2))
        for t in range(t_birth + 1, t_death):
            X[t, i, :] = np.random.multivariate_normal(F @ X[t - 1, i, :], Q)
    return X[:, :, (0, 2)]

def add_track_switching(X, n_switches, switch_cutoff, max_switch_attempts):
    X = X.copy()
    switches = 0
    attempted_switches = 0
    completed_switches = set()
    while switches < n_switches and attempted_switches < max_switch_attempts:
        t, i, j = np.random.randint(0, X.shape[0]), np.random.randint(0, X.shape[1]), np.random.randint(0, X.shape[1])
        if i == j:
            continue
        if np.linalg.norm(X[t, i, :] - X[t, j, :]) < switch_cutoff and (t, i, j) not in completed_switches:
            tmp = X[t:, i, :].copy()
            X[t:, i, :] = X[t:, j, :].copy()
            X[t:, j, :] = tmp
            switches += 1
            completed_switches.add((t, i, j))
            completed_switches.add((t, j, i))
        attempted_switches += 1
    return X
    
def generate_structured_data(T, m_t, m_f, n_f, l, switch_cutoff, n_switches, max_switch_attempts, noise, birth_death_prob=0.5):
    X_t = generate_trajectories(T, m_t, l, birth_death_prob)
    Y_t = add_track_switching(X_t, n_switches, switch_cutoff, max_switch_attempts)
    X_f = generate_trajectories(T, m_f, l, birth_death_prob)
    Y_f = generate_trajectories(T, n_f, l, birth_death_prob)
    X = np.concatenate((X_t, X_f), axis=1)
    Y = np.concatenate((Y_t, Y_f), axis=1)
    Y += np.random.normal(0, noise, size=Y.shape)
    return X, Y

# Linear programming implementation of the T-GOSPA metric

In [ ]:
def linear_programming_solve(D, p, gamma, W_type=pulp.LpContinuous, logging=0):
    if logging >= 1:
        print("Setting up problem!")
    
    T, m, n = D.shape
    m, n = m - 1, n - 1

    prob = pulp.LpProblem("GOSPA-T", pulp.LpMinimize)

    time_steps = list(range(T))
    ground_truths = list(range(m))
    estimates = list(range(n))

    W = pulp.LpVariable.dicts("Assignment", (time_steps, ground_truths + [m], estimates + [n]), 0, 1, W_type)
    H = pulp.LpVariable.dicts("Track switching", (time_steps[:-1], ground_truths, estimates), None, None, pulp.LpContinuous)

    prob += (
        pulp.lpSum([D[t, i, j] * W[t][i][j] for t in time_steps for i in ground_truths + [m] for j in estimates + [n]]) +
            pulp.lpSum([gamma**p / 2 * H[t][i][j] for t in time_steps[:-1] for i in ground_truths for j in estimates]),
        "Sum of assignment and track switching cost",
    )

    for t in time_steps:
        for i in ground_truths:
            prob += (
                pulp.lpSum([W[t][i][j] for j in estimates + [n]]) == 1,
                f"Time step {t}, row {i} sum constraint",
            )
        for j in estimates:
            prob += (
                pulp.lpSum([W[t][i][j] for i in ground_truths + [m]]) == 1,
                f"Time step {t}, column {j} sum constraint",
            )
        prob += W[t][m][n] == 0
        
    for t in time_steps[:-1]:
        for i in ground_truths:
            for j in estimates:
                prob += (
                    H[t][i][j] >= W[t + 1][i][j] - W[t][i][j],
                    f"Time step {t}, element {i}, {j} track switching constraint (positive)",
                )
                prob += (
                    H[t][i][j] >= W[t][i][j] - W[t + 1][i][j],
                    f"Time step {t}, element {i}, {j} track switching constraint (negative)",
                )
                
    if logging >= 1:
        print("Starting solve!")
        
    status = prob.solve(pulp.PULP_CBC_CMD(msg=(logging == 2)))
    objective_value = pulp.value(prob.objective)**(1/p)
    node_cost = pulp.value(pulp.lpSum([D[t, i, j] * W[t][i][j] for t in time_steps for i in ground_truths + [m] for j in estimates + [n]]))
    edge_cost = pulp.value(pulp.lpSum([gamma**p / 2 * H[t][i][j] for t in time_steps[:-1] for i in ground_truths for j in estimates]))
    
    W = np.array([[[W[t][i][j].varValue for i in range(m+1)] for j in range(n+1)] for t in range(T)])
    H = np.array([[[H[t][i][j].varValue for i in range(m)] for j in range(n)] for t in range(T-1)])

    if logging >= 1:
        print("Done!")
        
    return objective_value, node_cost, edge_cost, W, H, status

# Algorithm 1

In [ ]:
def algorithm_1_solve(D, p, gamma, eta, rho, norm_type=float('inf'), logging=False, record_objective=False, record_convergence=False, max_its=torch.inf):
    T, m, n = D.shape
    m, n = m - 1, n - 1
    
    mu_x = torch.ones(m + 1)
    mu_x[m] = n
    mu_y = torch.ones(n + 1)
    mu_y[n] = m

    u_x = torch.zeros((T, m + 1))
    u_y = torch.zeros((T, n + 1))

    psi = torch.zeros((T, m + 1, n + 1))
    psi_hat = torch.zeros((T, m + 1, n + 1))

    eps = eta * torch.maximum(torch.max(D), torch.tensor(gamma**p)) * T
    k = -D / eps

    if logging:
        print("Chosen epsilon:", eps)
    
    U = torch.zeros((T - 1, m + 1, n + 1))
    U_tilde = -U
    
    def bimarginal_projection(t):
        return psi_hat[t, :, :] + psi[t, :, :] 
        
    def quadmarginal_projection(t):
        if t == T - 1:
            return psi_hat[T - 1, :, :] + psi[T - 1, :, :]
        else:
            return u_x[t, :, None] + psi[t + 1, :, :] + psi_hat[t, :, :] + k[t, :, :] + u_y[t, :] + U[t, :, :] + U_tilde[t, :, :]
    
    def update_psi(t):
        if t == T - 1:
            psi[T - 1, :, :] = u_x[T - 1, :, None] + k[T - 1, :, :] + u_y[T - 1, :]
        else:
            psi[t, :, :] = torch.logsumexp(psi[t + 1, :, :] + U_tilde[t, :, :], dim=(0, 1)) + u_x[t, :, None] + k[t, :, :] + U[t, :, :] + u_y[t, :]

    def update_psi_hat(t):
        if t == 0:
            psi_hat[t, :, :] = 0
        else:
            psi_hat[t, :, :] = torch.logsumexp(u_x[t - 1, :, None] + psi_hat[t - 1, :, :] + k[t - 1, :, :] + U[t - 1, :, :] + u_y[t - 1, :], dim=(0, 1)) + U_tilde[t - 1, :, :]

    def iterate_u():
        update_psi_hat(t)
        u_x[t, :] = u_x[t, :] + torch.log(mu_x) - torch.logsumexp(bimarginal_projection(t), axis=1)
        update_psi(t)
        u_y[t, :] = u_y[t, :] + torch.log(mu_y) - torch.logsumexp(bimarginal_projection(t), axis=0)
        
    def iterate_U():
        update_psi(t)
        update_psi_hat(t + 1)
        P = quadmarginal_projection(t)
        
        U_tilde[t, :m, :n] = torch.clip(U_tilde[t, :m, :n] + 0.5 * logsubexp(bimarginal_projection(t)[:m, :n], P[:m, :n]) - 0.5 * logsubexp(bimarginal_projection(t + 1)[:m, :n], P[:m, :n]), 0, gamma**p / eps)
        U[t, :m, :n] = -U_tilde[t, :m, :n]
        
        #U[t, :m, :n] = torch.clip(U[t, :m, :n] + 0.5 * logsubexp(bimarginal_projection(t)[:m, :n], P[:m, :n]) - 0.5 * logsubexp(bimarginal_projection(t + 1)[:m, :n], P[:m, :n]), 0, gamma**p / eps)#-gamma**p / eps, 0)
        #U_tilde[t, :m, :n] = -U[t, :m, :n]
        
        
    def compute_objective():
        node_cost = 0
        for t in range(T):
            node_cost += torch.sum(torch.exp(bimarginal_projection(t)) * D[t, :, :])
        edge_cost = 0
        for t in range(T - 1):
            edge_cost += gamma**p / 2 * torch.sum((torch.abs(torch.exp(bimarginal_projection(t + 1)[:m, :n]) - torch.exp(bimarginal_projection(t)[:m, :n]))))
        return node_cost, edge_cost

    def logsubexp(x, y):
        if torch.any(y > x):
            print("This should not happen! Try increasing floating-point precision!")
        else:
            return x + torch.log(1.0 - torch.exp(y - x))

    it_cnt = 0
    node_costs = []
    edge_costs = []
    convergence = []
    u = torch.cat((u_x.flatten(), u_y.flatten(), U.flatten(), U_tilde.flatten()))
    u_prev = torch.zeros(u.shape)
    relative_change_norm = torch.inf
    
    while relative_change_norm > rho and it_cnt < max_its:
        u_prev = torch.clone(u)
        for t in range(T - 1, -1, -1):
            update_psi(t)
        for t in range(T - 1):
            iterate_u()
            iterate_U()
        t = T - 1    
        iterate_u()
        
        u = torch.cat((u_x.flatten(), u_y.flatten(), U.flatten(), U_tilde.flatten()))
        relative_change_norm =  torch.linalg.vector_norm(u - u_prev, ord=norm_type) / torch.linalg.vector_norm(u_prev, ord=norm_type)
        it_cnt += 1
        
        if record_objective:
            node_cost, edge_cost = compute_objective()
            node_costs.append(node_cost)
            edge_costs.append(edge_cost)
            
        if record_convergence:
            convergence.append(relative_change_norm)
        
        if logging:
            if it_cnt == 1 or it_cnt % logging == 0:
                print("Iteration:", it_cnt)
                print("    Objective value:", node_cost + edge_cost)
                print("        Node cost:", node_cost)
                print("        Edge cost:", edge_cost)
                print("    Convergence:", relative_change_norm)
                print()
        
    if logging:
        print("Done!")
    
    node_cost, edge_cost = compute_objective()
    
    return (node_cost + edge_cost)**(1/p), node_cost, edge_cost, it_cnt, torch.tensor(node_costs), torch.tensor(edge_costs), torch.tensor(convergence)

# Algorithm 2

In [ ]:
def algorithm_2_solve(D, p, gamma, eta, rho, norm_type=float('inf'), logging=False, record_objective=False, record_convergence=False, max_its=torch.inf):
    T, m, n = D.shape
    m, n = m - 1, n - 1
    
    mu_x = torch.ones(m + 1)
    mu_x[m] = n
    mu_y = torch.ones(n + 1)
    mu_y[n] = m
    
    u = torch.zeros((T, m + 1))
    u_hat = torch.zeros((T, n + 1))
    
    psi = torch.zeros((T, m + 1, n + 1))
    psi_hat = torch.zeros((T, m + 1, n + 1))
    
    D_tilde = torch.ones((n+1, n+1)) - torch.diag(torch.ones(n+1))
    D_tilde[:n, n] = 1 / 2
    D_tilde[n, :n] = D_tilde[:n, n]
    D_tilde *= gamma**p
    
    eps = eta * torch.maximum(torch.max(D), torch.max(D_tilde)) * T
    k_tilde = -D_tilde / eps
    k = -D / eps
    
    if logging:
        print("Chosen epsilon:", eps)
    
    def iterate():
        P = u[t, :, None] + psi_hat[t, :, :] + k[t, :, :] + psi[t, :, :] + u_hat[t, :]
        u[t, :] = u[t, :] + torch.log(mu_x) - torch.logsumexp(P, axis=1)
        P = u[t, :, None] + psi_hat[t, :, :] + k[t, :, :] + psi[t, :, :] + u_hat[t, :]
        u_hat[t, :] = u_hat[t, :] + torch.log(mu_y) - torch.logsumexp(P, axis=0)
    
    def update_psi():
        psi[t, :m, :] = torch.logsumexp((psi[t + 1, :m, :] + k[t + 1, :m, :] + u[t + 1, :m, None] + u_hat[t + 1, None, :])[:, :, None] + k_tilde[None, :, :], axis=1)
        psi[t, m, :] = torch.logsumexp(psi[t + 1, m, :] + k[t + 1, m, :] + u[t + 1, m] + u_hat[t + 1, None, :], axis=(0, 1))
    
    def update_psi_hat():
        psi_hat[t + 1, :m, :] = torch.logsumexp((psi_hat[t, :m, :] + k[t, :m, :] + u[t, :m, None] + u_hat[t, None, :])[:, :, None] + k_tilde[None, :, :], axis=1)
        psi_hat[t + 1, m, :] = torch.logsumexp(psi_hat[t, m, :] + k[t, m, :] + u[t, m] + u_hat[t, None, :], axis=(0, 1))
    
    def compute_objective():
        node_cost = 0
        for t in range(T):
            P = u[t, :, None] + psi_hat[t, :, :] + k[t, :, :] + psi[t, :, :] + u_hat[t, :]
            node_cost += torch.sum(D[t, :, :] * torch.exp(P))
        edge_cost = 0
        for t in range(T - 1):
            for i in range(m):
                P = u_hat[t + 1, :, None] + psi[t + 1, i, :, None] + k[t + 1, i, :, None] + psi_hat[t, i, :] + k[t, i, :] + k_tilde + u_hat[t, :] + u[t, i] + u[t + 1, i]
                edge_cost += torch.sum(D_tilde * torch.exp(P))
        return node_cost, edge_cost
        
    it_cnt = 0
    node_costs = []
    edge_costs = []
    convergence = []
    u_concat = torch.cat((u.flatten(), u_hat.flatten()))
    u_concat_prev = torch.zeros(u_concat.shape)
    relative_change_norm = torch.inf
    
    while relative_change_norm > rho and it_cnt < max_its:
        u_concat_prev = torch.clone(u_concat)
        for t in range(T - 2, -1, -1):
            update_psi()
        for t in range(T - 1):
            iterate()
            update_psi_hat()
        t = T - 1
        iterate()
        
        u_concat = torch.cat((u.flatten(), u_hat.flatten()))
        relative_change_norm =  torch.linalg.vector_norm(u_concat - u_concat_prev, ord=norm_type) / torch.linalg.vector_norm(u_concat_prev, ord=norm_type)
        it_cnt += 1

        if record_objective:
            node_cost, edge_cost = compute_objective()
            node_costs.append(node_cost)
            edge_costs.append(edge_cost)
        
        if record_convergence:
            convergence.append(relative_change_norm)
        
        if logging:
            if it_cnt == 1 or it_cnt % logging == 0:
                print("Iteration:", it_cnt)
                print("    Objective value:", node_cost + edge_cost)
                print("        Node cost:", node_cost)
                print("        Edge cost:", edge_cost)
                print("    Convergence:", relative_change_norm)
                print()
        
    if logging:
        print("Done!")
    
    node_cost, edge_cost = compute_objective()
    
    return (node_cost + edge_cost)**(1/p), node_cost, edge_cost, it_cnt, torch.tensor(node_costs), torch.tensor(edge_costs), torch.tensor(convergence)

# Results: Convergence results

## Unstructured data

### Data generation

In [ ]:
torch.set_default_dtype(torch.double)
torch.manual_seed(0)
D = torch.rand((20, 17, 16))
D[:, -1, -1] = 0

### Linear programming reference

In [ ]:
objective_value_lp, node_cost_lp, edge_cost_lp, W, _, _ = linear_programming_solve(D, p=1, gamma=0.1, logging=1)
print("Objective value:", objective_value_lp)
print("    Node cost:", node_cost_lp)
print("    Edge cost:", edge_cost_lp)

### Algorithm 1

In [ ]:
D = D.to('cuda')
with torch.device('cuda'):
    objective_value, node_cost, edge_cost, it_cnt, node_costs, edge_costs, convergence = algorithm_1_solve(D, p=1, gamma=0.1, eta=0.5 * 1e-4, rho=1e-10, norm_type=2, \
                                                                                                           logging=100, record_objective=True, record_convergence=True, max_its=1000)
save_path = "experiments/convergence_unstructured_algorithm_1_"
torch.save(node_costs, save_path + "node_costs.pt")
torch.save(edge_costs, save_path + "edge_costs.pt")
torch.save(convergence, save_path + "convergence.pt")

In [ ]:
load_path = "experiments/convergence_unstructured_algorithm_1_"
node_costs = torch.load(load_path + "node_costs.pt")
edge_costs = torch.load(load_path + "edge_costs.pt")
convergence = torch.load(load_path + "convergence.pt")

In [ ]:
plt.rcParams.update({'font.size': 17})
relative_step_size = convergence.cpu()
plt.semilogy(relative_step_size, color = "b")
plt.grid()
plt.xlabel("Number of Iterations")
plt.ylabel("Relative Step Size")
plt.savefig("figures/convergence_unstructured_algorithm_1_relative_step_size.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
objective_values = np.abs((node_costs + edge_costs).cpu() / objective_value_lp - 1)
plt.semilogy(objective_values, color = "r")
plt.grid()
plt.xlabel("Number of Iterations")
plt.ylabel("Relative Error")
plt.ylim((0.7*1e-3, 1e-1))
plt.axhline(color="k", linestyle=":")
plt.savefig("figures/convergence_unstructured_algorithm_1_objective_value.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
print("Final approximation:", (node_costs + edge_costs).cpu()[-1])
print("LP-solver:", objective_value_lp)
torch.set_printoptions(sci_mode=True)
print("Relative error:", np.abs((node_costs + edge_costs).cpu()[-1] / objective_value_lp - 1))
torch.set_printoptions(sci_mode=None)
print("Percent error:", np.abs((node_costs + edge_costs).cpu()[-1] / objective_value_lp - 1) * 100)

### Algorithm 2

In [ ]:
D = D.to('cuda')
with torch.device('cuda'):
    objective_value, node_cost, edge_cost, it_cnt, node_costs, edge_costs, convergence = algorithm_2_solve(D, p=1, gamma=0.1, eta=0.5 * 1e-4, rho=1e-10, norm_type=2, \
                                                                                                           logging=100, record_objective=True, record_convergence=True, max_its=1000)
save_path = "experiments/convergence_unstructured_algorithm_2_"
torch.save(node_costs, save_path + "node_costs.pt")
torch.save(edge_costs, save_path + "edge_costs.pt")
torch.save(convergence, save_path + "convergence.pt")

In [ ]:
load_path = "experiments/convergence_unstructured_algorithm_2_"
node_costs = torch.load(load_path + "node_costs.pt")
edge_costs = torch.load(load_path + "edge_costs.pt")
convergence = torch.load(load_path + "convergence.pt")

In [ ]:
plt.rcParams.update({'font.size': 17})
relative_step_size = convergence.cpu()
plt.semilogy(relative_step_size, color = "b")
plt.grid()
plt.xlabel("Number of Iterations")
plt.ylabel("Relative Step Size")
plt.savefig("figures/convergence_unstructured_algorithm_2_relative_step_size.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
objective_values = np.abs((node_costs + edge_costs).cpu() / objective_value_lp - 1)
plt.semilogy(objective_values, color = "r")
plt.grid()
plt.xlabel("Number of Iterations")
plt.ylabel("Relative Error")
plt.axhline(color="k", linestyle=":")
plt.savefig("figures/convergence_unstructured_algorithm_2_objective_value.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
print("Final approximation:", (node_costs + edge_costs).cpu()[-1])
print("LP-solver:", objective_value_lp)
torch.set_printoptions(sci_mode=True)
print("Relative error:", np.abs((node_costs + edge_costs).cpu()[-1] / objective_value_lp - 1))
torch.set_printoptions(sci_mode=None)
print("Percent error:", np.abs((node_costs + edge_costs).cpu()[-1] / objective_value_lp - 1) * 100)

## Structured data

### Data generation

In [ ]:
torch.set_default_dtype(torch.double)
np.random.seed(0)
X, Y = generate_structured_data(T=20, m_t=14, m_f=2, n_f=1, l=1, switch_cutoff=0.25, n_switches=20, max_switch_attempts=100000, noise=0.01, birth_death_prob=0.9)
D = calculate_distance_matrices(X, Y, c=0.25, p=1)
D = torch.from_numpy(D)

In [ ]:
plt.plot(X[:, :, 1], X[:, :, 0], "r", label = "Ground Truth")
plt.plot(Y[:, :, 1], Y[:, :, 0], "b:", label = "Estimate")
plt.axis('scaled')
plt.grid()
plt.xlabel("Horizontal Position")
plt.ylabel("Vertical Position")
plt.xlim((None, 3.5))
plt.ylim((None, 3.5))
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.savefig("figures/structured_data_visualization.pdf", bbox_inches="tight")
plt.show()
plt.close()

### Linear programming reference

In [ ]:
objective_value_lp, node_cost_lp, edge_cost_lp, W, _, _ = linear_programming_solve(D, p=1, gamma=1, logging=1)
print("Objective value:", objective_value_lp)
print("    Node cost:", node_cost_lp)
print("    Edge cost:", edge_cost_lp)

### Algorithm 1

In [ ]:
D = D.to('cuda')
with torch.device('cuda'):
    objective_value, node_cost, edge_cost, it_cnt, node_costs, edge_costs, convergence = algorithm_1_solve(D, p=1, gamma=1, eta=1e-5, rho=1e-10, norm_type=2, \
                                                                                                           logging=100, record_objective=True, record_convergence=True, max_its=1000)
save_path = "experiments/convergence_structured_algorithm_1_"
torch.save(node_costs, save_path + "node_costs.pt")
torch.save(edge_costs, save_path + "edge_costs.pt")
torch.save(convergence, save_path + "convergence.pt")

In [ ]:
load_path = "experiments/convergence_structured_algorithm_1_"
node_costs = torch.load(load_path + "node_costs.pt")
edge_costs = torch.load(load_path + "edge_costs.pt")
convergence = torch.load(load_path + "convergence.pt")

In [ ]:
plt.rcParams.update({'font.size': 17})
relative_step_size = convergence.cpu()
plt.semilogy(relative_step_size, color = "b")
plt.grid()
plt.xlabel("Number of Iterations")
plt.ylabel("Relative Step Size")
plt.savefig("figures/convergence_structured_algorithm_1_relative_step_size.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
objective_values = np.abs((node_costs + edge_costs).cpu() / objective_value_lp - 1)
plt.semilogy(objective_values, color = "r")
plt.grid()
plt.xlabel("Number of Iterations")
plt.ylabel("Relative Error")
plt.axhline(color="k", linestyle=":")
plt.savefig("figures/convergence_structured_algorithm_1_objective_value.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
print("Final approximation:", (node_costs + edge_costs).cpu()[-1])
print("LP-solver:", objective_value_lp)
torch.set_printoptions(sci_mode=True)
print("Relative error:", np.abs((node_costs + edge_costs).cpu()[-1] / objective_value_lp - 1))
torch.set_printoptions(sci_mode=None)
print("Percent error:", np.abs((node_costs + edge_costs).cpu()[-1] / objective_value_lp - 1) * 100)

### Algorithm 2

In [ ]:
D = D.to('cuda')
with torch.device('cuda'):
    objective_value, node_cost, edge_cost, it_cnt, node_costs, edge_costs, convergence = algorithm_2_solve(D, p=1, gamma=1, eta=1e-5, rho=1e-10, norm_type=2, \
                                                                                                           logging=100, record_objective=True, record_convergence=True, max_its=1000)  
save_path = "experiments/convergence_structured_algorithm_2_"
torch.save(node_costs, save_path + "node_costs.pt")
torch.save(edge_costs, save_path + "edge_costs.pt")
torch.save(convergence, save_path + "convergence.pt")

In [ ]:
load_path = "experiments/convergence_structured_algorithm_2_"
node_costs = torch.load(load_path + "node_costs.pt")
edge_costs = torch.load(load_path + "edge_costs.pt")
convergence = torch.load(load_path + "convergence.pt")

In [ ]:
plt.rcParams.update({'font.size': 17})
relative_step_size = convergence.cpu()
plt.semilogy(relative_step_size, color = "b")
plt.grid()
plt.xlabel("Number of Iterations")
plt.ylabel("Relative Step Size")
plt.savefig("figures/convergence_structured_algorithm_2_relative_step_size.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
objective_values = np.abs((node_costs + edge_costs).cpu() / objective_value_lp - 1)
plt.semilogy(objective_values, color = "r")
plt.grid()
plt.xlabel("Number of Iterations")
plt.ylabel("Relative Error")
plt.axhline(color="k", linestyle=":")
plt.savefig("figures/convergence_structured_algorithm_2_objective_value.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
print("Final approximation:", (node_costs + edge_costs).cpu()[-1])
print("LP-solver:", objective_value_lp)
torch.set_printoptions(sci_mode=True)
print("Relative error:", np.abs((node_costs + edge_costs).cpu()[-1] / objective_value_lp - 1))
torch.set_printoptions(sci_mode=None)
print("Percent error:", np.abs((node_costs + edge_costs).cpu()[-1] / objective_value_lp - 1) * 100)

# Results: Regularization analysis

In [ ]:
torch.set_printoptions(sci_mode=None)
etas = [1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2, 1e-1, 3e-1]
final_objective_values = []

for eta in tqdm(etas):
    D = D.to('cuda')
    with torch.device('cuda'):
        objective_value, node_cost, edge_cost, it_cnt, node_costs, edge_costs, convergence = algorithm_2_solve(D, p=1, gamma=1, eta=eta, rho=1.5e-6, norm_type=2, \
                                                                                                               logging=100, record_objective=True, record_convergence=True, max_its=1e300)  
    final_objective_values.append(objective_value)
    
    save_path = "experiments/regularization_" + str(eta) + "_structured_algorithm_2_"
    torch.save(node_costs, save_path + "node_costs.pt")
    torch.save(edge_costs, save_path + "edge_costs.pt")
    torch.save(convergence, save_path + "convergence.pt")

torch.save(final_objective_values, "experiments/regularization_final_objective_values.pt")

In [ ]:
final_objective_values = torch.load("experiments/regularization_final_objective_values.pt")
final_objective_values = [ov.cpu() for ov in final_objective_values]

In [ ]:
log_scale_plot = True

if log_scale_plot:
    fig, ax = plt.subplots()
    ax.loglog(etas, np.abs(torch.tensor(final_objective_values) / objective_value_lp - 1), "go-")
    ax.invert_xaxis()
    ax.grid()
    ax.set_ylabel("Relative Error")
    ax.set_xlabel("Regularization Magnitude")
    fig.savefig("figures/regularization_analysis_algorithm_2.pdf", bbox_inches="tight")
    plt.show()
    plt.close()
else:     
    fig, ax = plt.subplots()
    left, bottom, width, height = [0.5, 0.5, 0.4, 0.4]
    axins = ax.inset_axes([left, bottom, width, height])
    axins.grid()
    axins.set_ylim((-0.002, 0.04))
    axins.set_xlim((1e-3*1.2, 1e-5*0.8))
    axins.semilogx(etas[:5], np.abs(torch.tensor(final_objective_values) / objective_value_lp - 1)[:5], "go-")
    ax.axhline(color="k", linestyle=":")
    ax.semilogx(etas, np.abs(torch.tensor(final_objective_values) / objective_value_lp - 1), "go-")
    ax.invert_xaxis()
    ax.grid()
    axins.invert_xaxis()
    ax.indicate_inset_zoom(axins, edgecolor="black", zorder=0)
    axins.invert_xaxis()
    ax.set_ylabel("Relative Error")
    ax.set_xlabel("Regularization Magnitude")
    fig.savefig("figures/regularization_analysis_algorithm_2.pdf", bbox_inches="tight")
    plt.show()
    plt.close()

# Results: Runtime analysis

## Varying number of trajectories

In [ ]:
torch.set_default_dtype(torch.float)
np.random.seed(0)
clk_id = time.CLOCK_MONOTONIC

ms = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
ms.reverse()
m_fs = [round(i / 10) for i in ms]
m_ts = [ms[i] - m_fs[i] for i in range(len(ms))]
Ts = [25 for _ in range(len(ms))]

n_samples = 50
printing = False

timing_lp = np.zeros((n_samples, len(ms)))
timing_alg_2_cpu = np.zeros((n_samples, len(ms)))
error_alg_2_cpu = np.zeros((n_samples, len(ms)))
timing_alg_2_gpu = np.zeros((n_samples, len(ms)))
error_alg_2_gpu = np.zeros((n_samples, len(ms)))

for i in trange(n_samples):
    if printing:
        print(f"Sample {i+1}:")
    for j in range(len(ms)):
        if printing:
            print(f"    Case: T = {Ts[j]}, m = {ms[j]}:")
        X, Y = generate_structured_data(T=Ts[j], m_t=m_ts[j], m_f=m_fs[j], n_f=m_fs[j], l=1, switch_cutoff=0.25, n_switches=ms[j], max_switch_attempts=100 * ms[j], noise=0.01, birth_death_prob=0.9)
        D = calculate_distance_matrices(X, Y, c=0.25, p=1)
        D = torch.from_numpy(D).float()
        start_time = time.clock_gettime(clk_id)
        objective_value_lp, node_cost_lp, edge_cost_lp, W, _, _ = linear_programming_solve(D, p=1, gamma=1, logging=False)
        elapsed_time = time.clock_gettime(clk_id) - start_time
        timing_lp[i, j] = elapsed_time
        if printing:
            print("        Linear programming:")
            print("            Objective value:", objective_value_lp)
            print("                Node cost:", node_cost_lp)
            print("                Edge cost:", edge_cost_lp)
            print("            Time:", elapsed_time)

        start_time = time.clock_gettime(clk_id)
        objective_value, node_cost, edge_cost, it_cnt, node_costs, edge_costs, convergence = algorithm_2_solve(D, p=1, gamma=1, eta=1e-4, rho=1e-4, norm_type=2, logging=False, record_objective=False, record_convergence=False, max_its=1e300)
        elapsed_time = time.clock_gettime(clk_id) - start_time
        timing_alg_2_cpu[i, j] = elapsed_time
        error_alg_2_cpu[i, j] = np.abs(objective_value / objective_value_lp - 1)
        if printing:
            print("        Algorithm 2 (CPU):")
            print("            Objective value:", objective_value)
            print("            Relative error:", error_alg_2_cpu[i, j])
            print("            Time:", elapsed_time)
        
        start_time = time.clock_gettime(clk_id)
        D = D.to('cuda')
        with torch.device('cuda'):
            objective_value, node_cost, edge_cost, it_cnt, node_costs, edge_costs, convergence = algorithm_2_solve(D, p=1, gamma=1, eta=1e-4, rho=1e-4, norm_type=2, logging=False, record_objective=False, record_convergence=False, max_its=1e300)
        elapsed_time = time.clock_gettime(clk_id) - start_time
        timing_alg_2_gpu[i, j] = elapsed_time
        error_alg_2_gpu[i, j] = np.abs(objective_value.cpu() / objective_value_lp - 1)
        if printing:
            print("        Algorithm 2 (GPU):")
            print("            Objective value:", objective_value)
            print("            Relative error:", error_alg_2_gpu[i, j])
            print("            Time:", elapsed_time)
        
save_path = "experiments/runtime_targets_lp_"
torch.save(timing_lp, save_path + "timing_lp.pt")
torch.save(timing_alg_2_cpu, save_path + "timing_alg_2_cpu.pt")
torch.save(error_alg_2_cpu, save_path + "error_alg_2_cpu.pt")
torch.save(timing_alg_2_gpu, save_path + "timing_alg_2_gpu.pt")
torch.save(error_alg_2_gpu, save_path + "error_alg_2_gpu.pt")

In [ ]:
load_path = "experiments/runtime_targets_lp_"
timing_lp = torch.load(load_path + "timing_lp.pt")
timing_alg_2_cpu = torch.load(load_path + "timing_alg_2_cpu.pt")
error_alg_2_cpu = torch.load(load_path + "error_alg_2_cpu.pt")
timing_alg_2_gpu = torch.load(load_path + "timing_alg_2_gpu.pt")
error_alg_2_gpu = torch.load(load_path + "error_alg_2_gpu.pt")

In [ ]:
ms.reverse()

plt.plot(ms, timing_lp.mean(axis=0)[::-1], "b-", linewidth=1, marker=".", label="LP-solver")
plt.plot(ms, timing_alg_2_cpu.mean(axis=0)[::-1], "r-", linewidth=1, marker=".", label="Algorithm 2 (CPU)")
plt.plot(ms, timing_alg_2_gpu.mean(axis=0)[::-1], "g-", linewidth=1, marker=".", label="Algorithm 2 (GPU)")
plt.legend()
plt.grid()
plt.xlabel("Number of targets")
plt.ylabel("Wall-clock time in seconds")
plt.savefig("figures/runtime_targets_timings.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
mean = error_alg_2_cpu.mean(axis=0)[::-1]
maxs = error_alg_2_cpu.max(axis=0)[::-1]
std = error_alg_2_cpu.std(axis=0)[::-1]

plt.plot(ms, mean, "r-", linewidth=1, marker=".", label="Mean")
plt.plot(ms, maxs, "k-", linewidth=1, marker=".", label="Max")
plt.legend()
plt.fill_between(ms, mean - std, mean + std, alpha=0.1, color = "r")
plt.grid()
plt.xlabel("Number of targets")
plt.ylabel("Relative error")
plt.savefig("figures/runtime_targets_error.pdf", bbox_inches="tight")
plt.show()
plt.close()

## Varying number of time steps

In [ ]:
torch.set_default_dtype(torch.float)
np.random.seed(0)
clk_id = time.CLOCK_MONOTONIC

Ts = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
ms = [30 for _ in range(len(Ts))]
m_fs = [round(i / 10) for i in ms]
m_ts = [ms[i] - m_fs[i] for i in range(len(Ts))]
Ts.reverse()

n_samples = 50
printing = False

timing_lp = np.zeros((n_samples, len(Ts)))
timing_alg_2_cpu = np.zeros((n_samples, len(Ts)))
error_alg_2_cpu = np.zeros((n_samples, len(Ts)))
timing_alg_2_gpu = np.zeros((n_samples, len(Ts)))
error_alg_2_gpu = np.zeros((n_samples, len(Ts)))

for i in trange(n_samples):
    if printing:
        print(f"Sample {i+1}:")
    for j in range(len(Ts)):
        if printing:
            print(f"    Case: T = {Ts[j]}, m = {ms[j]}:")
        X, Y = generate_structured_data(T=Ts[j], m_t=m_ts[j], m_f=m_fs[j], n_f=m_fs[j], l=1, switch_cutoff=0.25, n_switches=ms[j], max_switch_attempts=100 * ms[j], noise=0.01, birth_death_prob=0.9)
        D = calculate_distance_matrices(X, Y, c=0.25, p=1)
        D = torch.from_numpy(D).float()
        start_time = time.clock_gettime(clk_id)
        objective_value_lp, node_cost_lp, edge_cost_lp, W, _, _ = linear_programming_solve(D, p=1, gamma=1, logging=False)
        elapsed_time = time.clock_gettime(clk_id) - start_time
        timing_lp[i, j] = elapsed_time
        if printing:
            print("        Linear programming:")
            print("            Objective value:", objective_value_lp)
            print("                Node cost:", node_cost_lp)
            print("                Edge cost:", edge_cost_lp)
            print("            Time:", elapsed_time)

        start_time = time.clock_gettime(clk_id)
        objective_value, node_cost, edge_cost, it_cnt, node_costs, edge_costs, convergence = algorithm_2_solve(D, p=1, gamma=1, eta=1e-4, rho=1e-4, norm_type=2, logging=False, record_objective=False, record_convergence=False, max_its=1e300)
        elapsed_time = time.clock_gettime(clk_id) - start_time
        timing_alg_2_cpu[i, j] = elapsed_time
        error_alg_2_cpu[i, j] = np.abs(objective_value / objective_value_lp - 1)
        if printing:
            print("        Algorithm 2 (CPU):")
            print("            Objective value:", objective_value)
            print("            Relative error:", error_alg_2_cpu[i, j])
            print("            Time:", elapsed_time)
        
        start_time = time.clock_gettime(clk_id)
        D = D.to('cuda')
        with torch.device('cuda'):
            objective_value, node_cost, edge_cost, it_cnt, node_costs, edge_costs, convergence = algorithm_2_solve(D, p=1, gamma=1, eta=1e-4, rho=1e-4, norm_type=2, logging=False, record_objective=False, record_convergence=False, max_its=1e300)
        elapsed_time = time.clock_gettime(clk_id) - start_time
        timing_alg_2_gpu[i, j] = elapsed_time
        error_alg_2_gpu[i, j] = np.abs(objective_value.cpu() / objective_value_lp - 1)
        if printing:
            print("        Algorithm 2 (GPU):")
            print("            Objective value:", objective_value)
            print("            Relative error:", error_alg_2_gpu[i, j])
            print("            Time:", elapsed_time)
        
save_path = "experiments/runtime_time_steps_lp_"
torch.save(timing_lp, save_path + "timing_lp.pt")
torch.save(timing_alg_2_cpu, save_path + "timing_alg_2_cpu.pt")
torch.save(error_alg_2_cpu, save_path + "error_alg_2_cpu.pt")
torch.save(timing_alg_2_gpu, save_path + "timing_alg_2_gpu.pt")
torch.save(error_alg_2_gpu, save_path + "error_alg_2_gpu.pt")

In [ ]:
load_path = "experiments/runtime_time_steps_lp_"
timing_lp = torch.load(load_path + "timing_lp.pt")
timing_alg_2_cpu = torch.load(load_path + "timing_alg_2_cpu.pt")
error_alg_2_cpu = torch.load(load_path + "error_alg_2_cpu.pt")
timing_alg_2_gpu = torch.load(load_path + "timing_alg_2_gpu.pt")
error_alg_2_gpu = torch.load(load_path + "error_alg_2_gpu.pt")

In [ ]:
Ts.reverse()

plt.plot(Ts, timing_lp.mean(axis=0)[::-1], "b-", linewidth=1, marker=".", label="LP-Solver")
plt.plot(Ts, timing_alg_2_cpu.mean(axis=0)[::-1], "r-", linewidth=1, marker=".", label="Algorithm 2 (CPU)")
plt.plot(Ts, timing_alg_2_gpu.mean(axis=0)[::-1], "g-", linewidth=1, marker=".", label="Algorithm 2 (GPU)")
plt.legend()
plt.grid()
plt.xlabel("Number of Time Steps")
plt.ylabel("Wall-Clock Time in Seconds")
plt.savefig("figures/runtime_time_steps_timings.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
mean = error_alg_2_cpu.mean(axis=0)[::-1]
maxs = error_alg_2_cpu.max(axis=0)[::-1]
std = error_alg_2_cpu.std(axis=0)[::-1]

plt.plot(Ts, mean, "r-", linewidth=1, marker=".", label="Mean")
plt.plot(Ts, maxs, "k-", linewidth=1, marker=".", label="Max")
plt.legend()
plt.fill_between(Ts, mean - std, mean + std, alpha=0.1, color = "r")
plt.grid()
plt.xlabel("Number of Time Steps")
plt.ylabel("Relative Error")
plt.savefig("figures/runtime_time_steps_error.pdf", bbox_inches="tight")
plt.show()
plt.close()

# Counterexamples to the integrality of the linear program

In [ ]:
# Counterexample:
# X = np.array([[[0.5, 0.5, 1. , 3. ], [1.5, 0.5, 4. , 1.5]]])
# Y = np.array([[[1.5, 0.5, 3. , 3. ], [1.5, 4.5, 4.5, 1. ]]])
# c = 5.5
# gamma = 0.5


# Counterexample:
# X = np.array([[[0.5, 0. , 4.5], [1. , 2. , 4. ]]])
# Y = np.array([[[2. , 3. , 0.5], [0. , 0.5, 4. ]]])
# c = 2
# gamma = 1


# Counterexample:
# X = np.array([[[2.5, 0.5, 4. ], [0. , 0.5, 4.5]]])
# Y = np.array([[[4.5, 4. ], [1.5, 0.5]]])
# c = 2
# gamma = 1

In [ ]:
#def generate_counter_example(T, m, n, roof):
#    objective_value_continuous, objective_value_binary = 0, 0
#    while True:
#        X = np.random.randint(roof, size=(T, m, 1)) / 2
#        Y = np.random.randint(roof, size=(T, m, 1)) / 2
#        c = np.random.randint(roof) / 2 + 1
#        gamma = np.random.randint(roof) / 2
#        D = calculate_distance_matrices(X, Y, c, gamma)
#        objective_value, node_cost, edge_cost, W, H, status = linear_programming_solve(D, 1, gamma, W_type=pulp.LpContinuous, logging=0)
#        b_objective_value, b_node_cost, b_edge_cost, b_W, b_H, b_status = linear_programming_solve(D, 1, gamma, W_type=pulp.LpBinary, logging=0)
#        print(objective_value, b_objective_value)
#        if b_objective_value != objective_value:
#            return X, Y, c, gamma

#X, Y, c, gamma = generate_counter_example(10, 10, 10, 10)

In [ ]:
#X = np.array([[[2.5, 0.5, 4. ], [0. , 0.5, 4.5]]])
#Y = np.array([[[4.5, 4. ], [1.5, 0.5]]])
#gamma = 1

#X = np.transpose(X, axes=(1, 2, 0))
#Y = np.transpose(Y, axes=(1, 2, 0))

#D = calculate_distance_matrices(X, Y, c, gamma)
#objective_value, node_cost, edge_cost, W, H, status = linear_programming_solve(D, 1, gamma, W_type=pulp.LpContinuous, logging=0)
#b_objective_value, b_node_cost, b_edge_cost, b_W, b_H, b_status = linear_programming_solve(D, 1, gamma, W_type=pulp.LpBinary, logging=0)
#print(objective_value, b_objective_value)

#print(W)
#print(H)

#print(b_W)
#print(b_H)

#print(pulp.LpStatus[status])
#print(pulp.LpStatus[b_status])

# Illustration of Tracking Errors

In [ ]:
def gospa(D, W_type):
    T, m, n = D.shape
    m, n = m - 1, n - 1

    prob = pulp.LpProblem("GOSPA", pulp.LpMinimize)

    time_steps = list(range(T))
    ground_truths = list(range(m))
    estimates = list(range(n))

    W = pulp.LpVariable.dicts("Assignment", (time_steps, ground_truths + [m], estimates + [n]), 0, 1, W_type)

    prob += (
        pulp.lpSum([D[t, i, j] * W[t][i][j] for t in time_steps for i in ground_truths + [m] for j in estimates + [n]]),
        "Assignment cost",
    )

    for t in time_steps:
        for i in ground_truths:
            prob += (
                pulp.lpSum([W[t][i][j] for j in estimates + [n]]) == 1,
                f"Time step {t}, row {i} sum constraint",
            )
        for j in estimates:
            prob += (
                pulp.lpSum([W[t][i][j] for i in ground_truths + [m]]) == 1,
                f"Time step {t}, column {j} sum constraint",
            )
        prob += W[t][m][n] == 0

    status = prob.solve(pulp.PULP_CBC_CMD(msg=False))
    objective_value = pulp.value(prob.objective)
    W = np.array([[[W[t][i][j].varValue for i in range(m+1)] for j in range(n+1)] for t in range(T)])

    return status, objective_value, W

In [ ]:
np.random.seed(15)
m = 7
n = 7
X = np.random.random_sample((1, m, 2))
Y = X + np.random.normal(0, 0.04, size=X.shape)
D = calculate_distance_matrices(X, Y, 0.3, 1)
status, gospa_val, W = gospa(D, pulp.LpBinary)

for i in range(m):
    for j in range(n):
        if W[0, j, i] > 0.99:
                line = [(X[0, i, 0], Y[0, j, 0]), (X[0, i, 1], Y[0, j, 1])]
                plt.plot(*line, "k", lw=1)

plt.ylim(0, 1)
plt.xlim(0, 1)
plt.plot(X[0, :, 0], X[0, :, 1], '+b', markersize=10, label='Ground Truth')
plt.plot(Y[0, :, 0], Y[0, :, 1], 'xr', markersize=10, label='Estimate')
plt.legend()
plt.xlabel("Horizontal Position")
plt.ylabel("Vertical Position")
plt.savefig("figures/localization_error.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
np.random.seed(6)
m = 5
n = 5
X = np.random.random_sample((1, m, 2))
Y = X + np.random.normal(0, 0.02, size=X.shape)
D = calculate_distance_matrices(X, Y, 0.3, 1)
status, gospa_val, W = gospa(D, pulp.LpBinary)

X = np.concatenate((X, np.random.random_sample((1, 2, 2))), axis=1)
Y = np.concatenate((Y, np.random.random_sample((1, 1, 2))), axis=1)

plt.ylim(0, 1)
plt.xlim(0, 1)

plt.plot(X[0, m:, 0], X[0, m:, 1], 'ko', markersize=20, mfc='w')
plt.plot(Y[0, n:, 0], Y[0, n:, 1], 'ko', markersize=20, mfc='w')
plt.plot(X[0, :, 0], X[0, :, 1], '+b', markersize=10, label='Ground Truth')
plt.plot(Y[0, :, 0], Y[0, :, 1], 'xr', markersize=10, label='Estimate')
plt.legend(loc=3)
plt.xlabel("Horizontal Position")
plt.ylabel("Vertical Position")
plt.savefig("figures/cardinality_error.pdf", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
X, Y = generate_simple_example(0.2, 2)

plt.vlines(range(1, 5), -1.5, 1.5, 'k', 'dotted')

plt.plot(range(1, 5), X[:, 0, 0], 'ob-', label='Ground\nTruth')
plt.plot(range(1, 5), X[:, 1, 0], 'ob-')

plt.plot(range(1, 5), Y[:, 0, 0], 'sr-', label='Estimate')
plt.plot(range(1, 5), Y[:, 1, 0], 'sr-')
plt.legend(loc=0)
plt.xlabel('Time')
plt.ylabel("Position")
plt.xticks(range(1, 5))
plt.savefig("figures/track_switching_error.pdf", bbox_inches="tight")
plt.show()
plt.close()